<a href="https://colab.research.google.com/github/prasvijaya/datascienceportfolio/blob/master/Topic_Modelling_using_LDA_gensim_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
data= pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', delimiter=',', quotechar='"')
data_text= data[['Consumer complaint narrative']]
data_text['index']= data_text.index
documents= data_text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
#Shape of dataset
print(len(documents))

57453


In [4]:
pd.set_option('display.max_colwidth', -1)
documents.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Consumer complaint narrative,index
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",0
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",1
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",2
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",3
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never stopped it from being reported when I specifically told them not to, due to the circumstances above.",4


In [5]:
#Importing gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stemmer= SnowballStemmer('english')

#function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
      if token not in gensim.parsing.preprocessing.STOPWORDS:
        if len(word) > 3 and len(word.strip('Xx/')) > 2:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
#preview the document
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['On', 'XX/XX/2019', 'i', 'received', 'an', 'summons.', 'i', 'was', 'being', 'sued', 'by', 'medical', 'recovery', 'services.', 'On', 'XX/XX/2019', 'i', 'personal', 'went', 'to', 'the', 'office', 'of', 'XXXX', 'XXXX', 'XXXX', 'XXXX', 'XXXX', 'XXXX,', 'pllc.', 'They', 'explained', 'the', 'debt', 'i', 'owed', 'and', 'the', 'fact', 'that', 'I', 'now', 'owe', 'filing', 'fees,', 'attorneys', 'fees,', 'and', 'legal', 'interest.', 'I', 'paid', '{$630.00}', 'for', 'everything.', 'On', 'XX/XX/2019', 'my', 'wife', 'filed', 'chapter', 'XXXX', 'bankruptcy.', '\nOn', 'XX/XX/2019', 'i', 'received', 'an', 'email', 'from', 'XXXX', '', 'saying', 'a', 'new', 'collection', 'account', 'was', 'added', 'to', 'my', 'credit.', 'After', 'reading', 'the', 'email', 'and', 'looking', 'at', 'my', 'credit', 'report.', 'I', 'already', 'paid', 'this', 'account.', 'On', 'XX/XX/2019', 'I', 'called', 'medical', 'recovery', 'services', 'and', 'they', 'refused', 'to', 'give', 'me', 'information', 'on', 

In [8]:
processed_docs = documents['Consumer complaint narrative'].map(preprocess)
processed_docs[:10]

0    [auto, loan, open, xx, xx, xxxx, nc, bb, north, carolina, resid, file, disput, twice, credit, bureaus, time, bb, claim, accur, loan, wasn, awar, today, tri, contact, bb, multipl, time, get, live, person, drive, own, car, knowledg, account, check, xxxxxxxx, xxxx, notic, ve, tri, twice, disput, addit, receiv, bill, inform, account, resort, tri, remov, fraudul, loan, account]                                                                                                                                                                                                                                                                                                                                                                 
1    [xxxx, notic, debt, credit, believ, think, speedi, cash, buy, old, debt, sell, xxxx, xxxx, xxxx, xxxx, contact, xxxx, xxxx, xxxx, xxxx, attempt, give, come, give, social, rep, say, account, pop, tell, account, els, continu, let, payment, payment, confirm, numbe

Bag of Words on the Data set

Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 account
1 accur
2 addit
3 auto
4 awar
5 bb
6 bill
7 bureaus
8 car
9 carolina
10 check


Gensim filter_extremes

Filter out tokens that appear in

    less than 15 documents (absolute number) or
    more than 0.5 documents (fraction of total corpus size, not absolute number).
    after the above two steps, keep only the first 100000 most frequent tokens.

In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(12, 3),
 (15, 2),
 (18, 1),
 (28, 1),
 (29, 2),
 (44, 1),
 (50, 3),
 (68, 1),
 (73, 1),
 (76, 2),
 (80, 1),
 (88, 2),
 (96, 1),
 (108, 1),
 (126, 2),
 (139, 2),
 (167, 1),
 (173, 1),
 (179, 1),
 (180, 1),
 (197, 1),
 (200, 3),
 (209, 1),
 (223, 2),
 (229, 1),
 (232, 1),
 (233, 1),
 (303, 2),
 (317, 1),
 (360, 3),
 (406, 2),
 (506, 1),
 (521, 1),
 (527, 2),
 (568, 1),
 (622, 1),
 (881, 1),
 (1259, 1),
 (1272, 1),
 (1365, 1),
 (1366, 1)]

In [12]:
#Preview Bag Of Words for our sample preprocessed document.

bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 12 ("credit") appears 3 time.
Word 15 ("file") appears 2 time.
Word 18 ("inform") appears 1 time.
Word 28 ("person") appears 1 time.
Word 29 ("receiv") appears 2 time.
Word 44 ("call") appears 1 time.
Word 50 ("debt") appears 3 time.
Word 68 ("payment") appears 1 time.
Word 73 ("report") appears 1 time.
Word 76 ("say") appears 2 time.
Word 80 ("tell") appears 1 time.
Word 88 ("go") appears 2 time.
Word 96 ("wife") appears 1 time.
Word 108 ("look") appears 1 time.
Word 126 ("collect") appears 2 time.
Word 139 ("medic") appears 2 time.
Word 167 ("sue") appears 1 time.
Word 173 ("attorney") appears 1 time.
Word 179 ("explain") appears 1 time.
Word 180 ("fact") appears 1 time.
Word 197 ("negat") appears 1 time.
Word 200 ("pay") appears 3 time.
Word 209 ("refus") appears 1 time.
Word 223 ("email") appears 2 time.
Word 229 ("legal") appears 1 time.
Word 232 ("month") appears 1 time.
Word 233 ("new") appears 1 time.
Word 303 ("servic") appears 2 time.
Word 317 ("add") appears 1 time.
Wor

**TF-IDF**

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [13]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.1263036629400697),
 (1, 0.08319183242243383),
 (2, 0.12229072314338907),
 (3, 0.1140015099475341),
 (4, 0.6442175089443114),
 (5, 0.08620082937914617),
 (6, 0.10224996425531692),
 (7, 0.11106267186406815),
 (8, 0.20548421419430446),
 (9, 0.058814551754840204),
 (10, 0.06867579820506955),
 (11, 0.04686450890959886),
 (12, 0.024698708808834447),
 (13, 0.12866918414699552),
 (14, 0.15201369480963434),
 (15, 0.0587400331454837),
 (16, 0.08714056814003608),
 (17, 0.06267965356558183),
 (18, 0.040262718087121015),
 (19, 0.12063719697547882),
 (20, 0.10569431379086351),
 (21, 0.15923358739598298),
 (22, 0.09329427211226869),
 (23, 0.19933125655617578),
 (24, 0.20758129637213243),
 (25, 0.07094928454072937),
 (26, 0.06898561131515528),
 (27, 0.14598426654660096),
 (28, 0.06744225220474716),
 (29, 0.03181707219623938),
 (30, 0.06934615126162556),
 (31, 0.14417072506009934),
 (32, 0.2172025020862819),
 (33, 0.0646219358060793),
 (34, 0.08085079972949236),
 (35, 0.15661786542993206),
 (36,

**Running LDA using Bag of Words**

Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"card" + 0.030*"charg" + 0.018*"disput" + 0.018*"credit" + 0.014*"receiv" + 0.012*"transact" + 0.011*"fraud" + 0.011*"purchas" + 0.010*"claim" + 0.009*"refund"
Topic: 1 
Words: 0.036*"bank" + 0.022*"check" + 0.020*"call" + 0.017*"tell" + 0.013*"number" + 0.013*"say" + 0.013*"money" + 0.011*"phone" + 0.011*"day" + 0.010*"time"
Topic: 2 
Words: 0.041*"credit" + 0.036*"card" + 0.020*"fee" + 0.016*"balanc" + 0.016*"bank" + 0.015*"payment" + 0.013*"charg" + 0.012*"well" + 0.012*"month" + 0.012*"fargo"
Topic: 3 
Words: 0.042*"loan" + 0.023*"mortgag" + 0.012*"servic" + 0.011*"payment" + 0.010*"home" + 0.009*"time" + 0.008*"receiv" + 0.008*"document" + 0.008*"request" + 0.008*"inform"
Topic: 4 
Words: 0.041*"debt" + 0.036*"report" + 0.034*"credit" + 0.026*"collect" + 0.016*"inform" + 0.015*"compani" + 0.011*"letter" + 0.011*"provid" + 0.010*"request" + 0.010*"send"
Topic: 5 
Words: 0.057*"payment" + 0.036*"pay" + 0.019*"call" + 0.019*"month" + 0.016*"tell" + 0.015*"time"

**Running LDA using TF-IDF**

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"mortgag" + 0.009*"loan" + 0.005*"properti" + 0.005*"report" + 0.005*"home" + 0.004*"modif" + 0.004*"payment" + 0.004*"tax" + 0.004*"credit" + 0.004*"servic"
Topic: 1 Word: 0.009*"bonus" + 0.008*"promot" + 0.007*"card" + 0.007*"car" + 0.007*"balanc" + 0.007*"credit" + 0.006*"charg" + 0.006*"offer" + 0.005*"open" + 0.005*"pay"
Topic: 2 Word: 0.018*"ident" + 0.018*"theft" + 0.009*"report" + 0.008*"victim" + 0.008*"belong" + 0.006*"number" + 0.006*"loan" + 0.006*"card" + 0.006*"debt" + 0.006*"call"
Topic: 3 Word: 0.010*"bank" + 0.009*"check" + 0.009*"fargo" + 0.009*"well" + 0.008*"card" + 0.006*"money" + 0.006*"close" + 0.005*"deposit" + 0.005*"tell" + 0.005*"charg"
Topic: 4 Word: 0.012*"payment" + 0.008*"card" + 0.008*"late" + 0.007*"fee" + 0.006*"charg" + 0.006*"credit" + 0.006*"month" + 0.006*"pay" + 0.006*"loan" + 0.005*"bank"
Topic: 5 Word: 0.019*"debt" + 0.013*"collect" + 0.011*"report" + 0.008*"compani" + 0.008*"credit" + 0.007*"owe" + 0.007*"valid" + 0.007*"le

**Performance evaluation by classifying sample document using LDA Bag of Words model**

In [17]:
processed_docs[4310]

['xx',
 'xx',
 'receiv',
 'summon',
 'sue',
 'medic',
 'recoveri',
 'servic',
 'xx',
 'xx',
 'person',
 'go',
 'offic',
 'xxxx',
 'xxxx',
 'xxxx',
 'xxxx',
 'xxxx',
 'xxxx',
 'pllc',
 'explain',
 'debt',
 'owe',
 'fact',
 'owe',
 'file',
 'fee',
 'attorney',
 'fee',
 'legal',
 'pay',
 'xx',
 'xx',
 'wife',
 'file',
 'chapter',
 'xxxx',
 'bankruptci',
 'xx',
 'xx',
 'receiv',
 'email',
 'xxxx',
 'say',
 'new',
 'collect',
 'account',
 'add',
 'credit',
 'read',
 'email',
 'look',
 'credit',
 'report',
 'pay',
 'account',
 'xx',
 'xx',
 'call',
 'medic',
 'recoveri',
 'servic',
 'refus',
 'inform',
 'collect',
 'pay',
 'fill',
 'desput',
 'xxxx',
 'tell',
 'owe',
 'debt',
 'debt',
 'caus',
 'negat',
 'impact',
 'credit',
 'month',
 'go',
 'say',
 'haven',
 'payment']

In [18]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5178062915802002	 
Topic: 0.041*"debt" + 0.036*"report" + 0.034*"credit" + 0.026*"collect" + 0.016*"inform" + 0.015*"compani" + 0.011*"letter" + 0.011*"provid" + 0.010*"request" + 0.010*"send"

Score: 0.47098323702812195	 
Topic: 0.057*"payment" + 0.036*"pay" + 0.019*"call" + 0.019*"month" + 0.016*"tell" + 0.015*"time" + 0.013*"compani" + 0.013*"say" + 0.011*"receiv" + 0.009*"credit"


Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

Performance evaluation by classifying sample document using LDA TF-IDF model

In [19]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.9859766960144043	 
Topic: 0.019*"debt" + 0.013*"collect" + 0.011*"report" + 0.008*"compani" + 0.008*"credit" + 0.007*"owe" + 0.007*"valid" + 0.007*"letter" + 0.006*"creditor" + 0.006*"disput"


**Testing model on unseen document**

In [20]:
unseen_document = 'They charged me interest on interest during moratorium period '
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5382722616195679	 Topic: 0.042*"loan" + 0.023*"mortgag" + 0.012*"servic" + 0.011*"payment" + 0.010*"home"
Score: 0.29415684938430786	 Topic: 0.032*"card" + 0.030*"charg" + 0.018*"disput" + 0.018*"credit" + 0.014*"receiv"
Score: 0.04224187880754471	 Topic: 0.041*"credit" + 0.036*"card" + 0.020*"fee" + 0.016*"balanc" + 0.016*"bank"
Score: 0.0418412871658802	 Topic: 0.057*"payment" + 0.036*"pay" + 0.019*"call" + 0.019*"month" + 0.016*"tell"
Score: 0.04178597405552864	 Topic: 0.041*"debt" + 0.036*"report" + 0.034*"credit" + 0.026*"collect" + 0.016*"inform"
Score: 0.04170180857181549	 Topic: 0.036*"bank" + 0.022*"check" + 0.020*"call" + 0.017*"tell" + 0.013*"number"


The text is correctly classified the topic as it belongs to Loan or Mortgage related concerns